# Week 7: Using Libraries - Part 2 (Advanced)

---

## Table of Contents
1. [Advanced tkinter Patterns](#advanced-tkinter)
   - MVC Architecture
   - Custom Widgets
   - Threading with GUIs
2. [Software Testing Pyramid](#testing-pyramid)
   - Unit Testing (Deep Dive)
   - Integration Testing
   - System Testing
   - Acceptance Testing
3. [Advanced unittest Features](#advanced-unittest)
   - Parameterized Tests
   - Mocking and Patching
   - Test Suites
4. [Alternative Testing Frameworks](#alt-testing)
   - pytest
   - doctest
5. [Exercises](#exercises)
6. [Homework](#homework)

---

## 1. Advanced tkinter Patterns <a name="advanced-tkinter"></a>

### **1.1 MVC Architecture with tkinter**
Separating concerns using Model-View-Controller pattern:

In [ ]:
import tkinter as tk

class Model:
    def __init__(self):
        self.data = ""
    
    def save(self, data):
        self.data = data
    
    def load(self):
        return self.data

class View(tk.Frame):
    def __init__(self, master):
        super().__init__(master)
        self.entry = tk.Entry(self)
        self.save_btn = tk.Button(self, text="Save")
        self.load_btn = tk.Button(self, text="Load")
        self.entry.pack()
        self.save_btn.pack()
        self.load_btn.pack()

class Controller:
    def __init__(self, root):
        self.model = Model()
        self.view = View(root)
        self.view.save_btn.config(command=self.save)
        self.view.load_btn.config(command=self.load)
        self.view.pack()
    
    def save(self):
        self.model.save(self.view.entry.get())
    
    def load(self):
        self.view.entry.delete(0, tk.END)
        self.view.entry.insert(0, self.model.load())

root = tk.Tk()
app = Controller(root)
root.mainloop()

### **1.2 Custom Widgets**
Creating reusable composite widgets:

In [ ]:
class LabeledEntry(tk.Frame):
    def __init__(self, parent, label_text, **kwargs):
        super().__init__(parent)
        self.label = tk.Label(self, text=label_text)
        self.entry = tk.Entry(self, **kwargs)
        self.label.pack(side=tk.LEFT)
        self.entry.pack(side=tk.RIGHT)
    
    def get(self):
        return self.entry.get()
    
    def insert(self, index, text):
        self.entry.insert(index, text)

# Usage
root = tk.Tk()
name_entry = LabeledEntry(root, "Name:", width=30)
name_entry.pack()
root.mainloop()

### **1.3 Threading with GUIs**
Preventing GUI freeze during long operations:

In [ ]:
import threading
import time

class LongTaskApp:
    def __init__(self, root):
        self.root = root
        self.button = tk.Button(root, text="Run Task", command=self.start_task)
        self.button.pack()
    
    def start_task(self):
        self.button.config(state=tk.DISABLED)
        threading.Thread(target=self.run_task, daemon=True).start()
    
    def run_task(self):
        # Simulate long-running task
        time.sleep(3)
        self.root.after(0, self.task_completed)
    
    def task_completed(self):
        self.button.config(state=tk.NORMAL)
        tk.messagebox.showinfo("Done", "Task completed!")

root = tk.Tk()
app = LongTaskApp(root)
root.mainloop()

---

## 2. Software Testing Pyramid <a name="testing-pyramid"></a>

```
        ___________________
       /                   \
      /    Acceptance      \
     /_____________________\
    /                       \
   /       System           \
  /_________________________\
 /                           \
/         Integration        \
\___________________________/
 \                         /
  \       Unit            /
   \____________________/
```

### **2.1 Unit Testing**
- Tests individual components in isolation
- Fast execution (milliseconds per test)
- 70-80% of all tests
- Example: Testing a single function

### **2.2 Integration Testing**
- Tests interactions between components
- Medium execution speed
- 15-20% of tests
- Example: Testing database interactions

### **2.3 System Testing**
- Tests complete system
- Slow execution
- 5-10% of tests
- Example: Testing API endpoints

### **2.4 Acceptance Testing**
- Validates business requirements
- Very slow (manual or automated UI tests)
- 1-5% of tests
- Example: Selenium tests

---

## 3. Advanced unittest Features <a name="advanced-unittest"></a>

### **3.1 Parameterized Tests**
Using `subTest` for data-driven testing:

In [ ]:
import unittest

class TestMath(unittest.TestCase):
    def test_multiply(self):
        test_cases = [
            (2, 3, 6),
            (0, 5, 0),
            (-1, 1, -1),
            (1.5, 2, 3.0)
        ]
        
        for a, b, expected in test_cases:
            with self.subTest(a=a, b=b):
                self.assertEqual(a * b, expected)

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

### **3.2 Mocking and Patching**
Isolating components during testing:

In [ ]:
from unittest.mock import patch, MagicMock

class Database:
    def get_user(self, user_id):
        # Actual database call
        pass

class TestUserService(unittest.TestCase):
    @patch('__main__.Database')
    def test_get_user(self, mock_db):
        # Configure mock
        mock_db.return_value.get_user.return_value = {'id': 1, 'name': 'Test'}
        
        # Test code that uses Database
        service = UserService(mock_db())
        user = service.get_user(1)
        
        self.assertEqual(user['name'], 'Test')
        mock_db.return_value.get_user.assert_called_once_with(1)

class UserService:
    def __init__(self, db):
        self.db = db
    
    def get_user(self, user_id):
        return self.db.get_user(user_id)

### **3.3 Test Suites**
Organizing tests into suites:

In [ ]:
def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestMath('test_multiply'))
    suite.addTest(TestUserService('test_get_user'))
    return suite

if __name__ == "__main__":
    runner = unittest.TextTestRunner()
    runner.run(suite())

---

## 4. Alternative Testing Frameworks <a name="alt-testing"></a>

### **4.1 pytest**
More concise syntax and powerful features:

In [ ]:
# pytest example (normally in separate test_*.py files)
def test_addition():
    assert 1 + 1 == 2

import pytest

@pytest.mark.parametrize("a,b,expected", [
    (1, 2, 3),
    (0, 0, 0),
    (-1, 1, 0)
])
def test_add(a, b, expected):
    assert a + b == expected

### **4.2 doctest**
Tests embedded in docstrings:

In [ ]:
def factorial(n):
    """
    Compute the factorial of n
    
    >>> factorial(0)
    1
    >>> factorial(5)
    120
    """
    return 1 if n == 0 else n * factorial(n-1)

if __name__ == "__main__":
    import doctest
    doctest.testmod()

---

## 5. Exercises <a name="exercises"></a>

1. **tkinter**: Create a custom widget combining:
   - Entry field
   - Validation label
   - Character counter

2. **Testing**: Write parameterized tests for:
   - Email validation function
   - Password strength checker
   - Using both unittest and pytest styles

---

## 6. Homework <a name="homework"></a>

1. **GUI Application**: Build a CSV viewer with:
   - File open dialog
   - Table display (use `ttk.Treeview`)
   - Search/filter functionality

2. **Test Suite**: Create a complete test suite for a blog system:
   - Unit tests for models
   - Integration tests for database layer
   - System tests for API endpoints
   - Use mocking where appropriate

---

## End of Week 7